In [1]:
import torch
from torch import nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt

from data_setup import create_dataloaders

from pathlib import Path

2.6.0+cu124 True


In [2]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

transforms = weights.transforms()

file_path = Path("mini_food/pizza_steak_sushi/")

train_path,test_path = file_path/"train",file_path/"test"


In [3]:
train_dataloader,test_dataloader,class_names = create_dataloaders(train_path,test_path,transforms,batch_size=32)

In [4]:
transforms,train_dataloader,test_dataloader,class_names

(ImageClassification(
     crop_size=[224]
     resize_size=[256]
     mean=[0.485, 0.456, 0.406]
     std=[0.229, 0.224, 0.225]
     interpolation=InterpolationMode.BICUBIC
 ),
 ['pizza', 'steak', 'sushi'])

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model = torchvision.models.efficientnet_b0().to(device=device)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [7]:
for param in model.features.parameters():
    param.requires_grad=False


In [8]:
from torchinfo import summary

summary(model=model,input_size=(1,3,256,256),col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 256, 256]     [1, 1000]            --                   Partial
├─Sequential (features)                                      [1, 3, 256, 256]     [1, 1280, 8, 8]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 256, 256]     [1, 32, 128, 128]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 256, 256]     [1, 32, 128, 128]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 128, 128]    [1, 32, 128, 128]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 128, 128]    [1, 32, 128, 128]    --                   --
│    └─Sequential (1)                                        [1, 32, 128, 128]    [1, 1

In [9]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [10]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,out_features=len(class_names),bias=True)
)

In [11]:
from torchinfo import summary

summary(model=model,input_size=(1,3,256,256),col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 256, 256]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 256, 256]     [1, 1280, 8, 8]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 256, 256]     [1, 32, 128, 128]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 256, 256]     [1, 32, 128, 128]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 128, 128]    [1, 32, 128, 128]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 128, 128]    [1, 32, 128, 128]    --                   --
│    └─Sequential (1)                                        [1, 32, 128, 128]    [1, 1

In [12]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [13]:
# !pip install tensorboard

In [14]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [15]:
device

'cuda'

In [20]:
def create_writer(experiment_name: str, 
                  model_name: str, 
                  extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter:
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if not Path("runs").exists():
        os.mkdir("runs")
    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
        
    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [21]:
example_writer_1 = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb0",
                               extra="5_epochs")
example_writer_2 = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb0",
                               extra="10_epochs")

[INFO] Created SummaryWriter, saving to: runs/2025-05-10/data_10_percent/effnetb0/5_epochs...
[INFO] Created SummaryWriter, saving to: runs/2025-05-10/data_10_percent/effnetb0/10_epochs...


In [22]:
from engine import train


model_0_results = train(model,optimizer,loss_fn,train_dataloader,test_dataloader,epochs=5,device=device,writer=example_writer_1)
model_1_results = train(model,optimizer,loss_fn,train_dataloader,test_dataloader,epochs=10,device=device,writer=example_writer_2)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1047 | train_acc: 30.4688 | test_loss: 1.0742 | test_acc: 38.2576
Epoch: 2 | train_loss: 1.1107 | train_acc: 41.4062 | test_loss: 1.1584 | test_acc: 26.0417
Epoch: 3 | train_loss: 1.1050 | train_acc: 30.0781 | test_loss: 1.2202 | test_acc: 30.1136
Epoch: 4 | train_loss: 1.1127 | train_acc: 31.2500 | test_loss: 1.1143 | test_acc: 31.1553
Epoch: 5 | train_loss: 1.0927 | train_acc: 45.7031 | test_loss: 1.1750 | test_acc: 28.1250


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1102 | train_acc: 42.9688 | test_loss: 1.1272 | test_acc: 31.1553
Epoch: 2 | train_loss: 1.1071 | train_acc: 29.6875 | test_loss: 1.1398 | test_acc: 35.2273
Epoch: 3 | train_loss: 1.1135 | train_acc: 48.0469 | test_loss: 1.1712 | test_acc: 31.1553
Epoch: 4 | train_loss: 1.0974 | train_acc: 33.2031 | test_loss: 1.1271 | test_acc: 32.1970
Epoch: 5 | train_loss: 1.0935 | train_acc: 41.4062 | test_loss: 1.1331 | test_acc: 30.2083
Epoch: 6 | train_loss: 1.0964 | train_acc: 41.0156 | test_loss: 1.1440 | test_acc: 32.1023
Epoch: 7 | train_loss: 1.0944 | train_acc: 31.6406 | test_loss: 1.1487 | test_acc: 27.0833
Epoch: 8 | train_loss: 1.0926 | train_acc: 45.7031 | test_loss: 1.1461 | test_acc: 31.1553
Epoch: 9 | train_loss: 1.0927 | train_acc: 31.6406 | test_loss: 1.1245 | test_acc: 28.1250
Epoch: 10 | train_loss: 1.1019 | train_acc: 32.0312 | test_loss: 1.0957 | test_acc: 32.1023


In [25]:
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir runs/2025-05-10/data_10_percent/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 45435), started 0:00:16 ago. (Use '!kill 45435' to kill it.)